### RNN Language model to process user events
Normal LSTM based model to evaluate the results of processing users
We are using tensorflow eager mode

In [1]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.8.0
Eager execution: True


In [2]:
user_names = ['U12', 'U13', 'U24', 'U78', 'U207', 'U293', 'U453', 'U679', 'U1289', 'U1480']
users_indir = '../data/users_feats'
users_lossdir = '../data/users_loss'
max_len = 120 # max length of sentence
num_chars = 128 # our vocabulary, i.e. unique characters in text. We'll just use the first 128 (half ASCII)

In [3]:
# transform character-based input into equivalent numerical versions
def encode_data(text, num_chars, max_length):
    # create empty vessels for one-hot encoded input
    X = np.zeros((len(text), max_length, num_chars), dtype=np.float32)
    y = np.zeros((len(text), max_length, num_chars), dtype=np.float32)
    
    # loop over inputs and tranform and store in X
    for i, sentence in enumerate(text):
        sentence = '\t' + sentence + '\n'
        for j, c in enumerate(sentence):
            X[i, j, ord(c)] = 1
            if j > 0:
                # target_data will be ahead by one timestep
                # and will not include the start character.
                y[i, j - 1, ord(c)] = 1.

    return X, y

In [4]:
def process_file(fname):
    """
        process file by extracting sentences data and encode them producing 
        a set of input and target data for processing by the model
        'fname' contains coma separated fields where the last one is the 
        sentence to be processes
    """
    data = open(fname).read()

    text = []
    red_events = []
    with open(fname, 'r') as infile:
        for line in infile.readlines():
            line = line.strip().split(',')
            text.append(line[-1])
            if int(line[2]) == 1:
                red_events.append(line)

#     print(text[0], 'len:', len(text[0]), len(text))

    input_data, target_data = encode_data(text, num_chars, max_len)
    
    return input_data, target_data, red_events
    

### Create a model using Keras

The TensorFlow [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras) API is the preferred way to create models and layers. This makes it easy to build models and experiment while Keras handles the complexity of connecting everything together. See the [Keras documentation](https://keras.io/) for details.

The [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model is a linear stack of layers. Its constructor takes a list of layer instances, in this case, one [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM) and one [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layers with 'num_chars' nodes each. The first layer's `input_shape` parameter corresponds to the amount of features from the dataset, and is required.

Our model will calculate its loss using the [tf.keras.losses.categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/categorical_crossentropy) function which takes the model's prediction and the desired output. The returned loss value is progressively larger as the prediction gets worse.

The `grad` function uses the `loss` function and the [tfe.GradientTape](https://www.tensorflow.org/api_docs/python/tf/contrib/eager/GradientTape) to record operations that compute the *[gradients](https://developers.google.com/machine-learning/crash-course/glossary#gradient)* used to optimize our model.

In [5]:

def getModel():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(max_len, input_shape=(None, num_chars), return_sequences=True),  # input shape required
#         tf.keras.layers.Dense(num_chars, activation="relu"),
        tf.keras.layers.Dense(num_chars, activation="softmax"),
    ])
    return model

def loss(model, x, y):
    y_ = model(x)
    return tf.keras.losses.categorical_crossentropy(y, y_)

def grad(model, inputs, targets):
    with tfe.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables)

#### Define the optimizer
This model uses the [tf.train.RMSPropOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer)  

In [6]:
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001, epsilon=1e-08)

### Model training

In [7]:
def train(model, training_dataset, num_epochs):
    
    train_losses = []
    loss_results = []
    
    for epoch in range(num_epochs):
        epoch_loss_avg = tfe.metrics.Mean()

        startTime = time.time()
        # training using batches of 'batch_size'
        for X, y in tfe.Iterator(training_dataset):
            grads = grad(model, X, y)
            optimizer.apply_gradients(zip(grads, model.variables), 
                                     global_step=tf.train.get_or_create_global_step())
            batch_loss = loss(model, X, y)
            epoch_loss_avg(batch_loss) # batch loss
            train_losses.append(tf.reduce_mean(batch_loss))

        loss_results.append(epoch_loss_avg.result())

        if epoch % 1 == 0:
            print("Epoch {:03d}: Loss: {:.3f} - in: {:.3f} sec.".format(epoch, 
                                                                        epoch_loss_avg.result(), 
                                                                        (time.time()-startTime)))        

    avg_loss = tf.reduce_mean(train_losses)
    max_loss = tf.reduce_max(train_losses)
    print('  avg_loss:', avg_loss, ' - max_loss:', max_loss)
 
    return loss_results

In [10]:
def process_user(user_name, outfile):
    num_epochs = 2
    batch_size = 500
    data_dir = '{0}/{1}/'.format(users_indir, u)
    
    # keep results for plotting
    train_loss_results = []

    model = getModel()

#     for i in range(57):
    for d in range(2):

        # training phase
        dataset_fname = data_dir+'{0}.txt'.format(d)
        print('df:', dataset_fname)
        input_data, target_data, red_events = process_file(dataset_fname)
        print('processing:', dataset_fname, " - num events:", len(input_data), " - red events:", len(red_events))

        training_dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data))
        training_dataset = training_dataset.batch(batch_size)

        # train model on a day
        loss_results = train(model, training_dataset, num_epochs)
        train_loss_results.append(loss_results)

        # evaluation phase
        dataset_fname = data_dir+'{0}.txt'.format(d+1)
        input_data, target_data, red_events = process_file(dataset_fname)
        print('  evaluating:', dataset_fname, " - num events:", len(input_data), " - red events:", len(red_events))

        eval_dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data))
        eval_dataset = eval_dataset.batch(batch_size)

        line_losses = np.array([])

        # eval using batches of 'batch_size'
        for X, y in tfe.Iterator(eval_dataset):
            line_losses = np.append(line_losses, tf.reduce_mean(loss(model, X, y), axis=1))

        possible_anomalies = [(i,v) for i, v in enumerate(line_losses)]
        possible_anomalies.sort(key=lambda x: x[1], reverse=True)
        
        print('    max:', possible_anomalies[:10])
        print('    red events:', [a for a,b in red_events])
        
        # write top 10 losses to a file with the format (day, score, redevent)
        for i, v in possible_anomalies[:20]:
            red = '0'
            for a,b in red_events:
                if a == i:
                    red = '1'
                    break
            line = '{0},{1},{2}\n'.format(d, v, red)
            outfile.write(line)
            

In [11]:

if not os.path.exists(users_lossdir):
    os.makedirs(users_lossdir)
    
for u in user_names:
    print('Processing files for User:', u)
    outfile_name = "{0}/{1}_losses.txt".format(users_lossdir, u)

    with open(outfile_name, 'w') as outfile:
        outfile.write('day,loss,redevent\n')
        process_user(u, outfile)
        outfile.close()

    

Processing files for User: U12
df: ../data/users_feats/U12/0.txt
processing: ../data/users_feats/U12/0.txt  - num events: 3128  - red events: 0
Epoch 000: Loss: 2.053 - in: 71.426 sec.
Epoch 001: Loss: 1.665 - in: 71.683 sec.
  avg_loss: tf.Tensor(1.8399007, shape=(), dtype=float32)  - max_loss: tf.Tensor(2.1780102, shape=(), dtype=float32)
  evaluating: ../data/users_feats/U12/1.txt  - num events: 8355  - red events: 0
    max: [(776, 1.825667142868042), (780, 1.825667142868042), (789, 1.825667142868042), (790, 1.825667142868042), (793, 1.825667142868042), (796, 1.825667142868042), (798, 1.825667142868042), (800, 1.825667142868042), (811, 1.825667142868042), (814, 1.825667142868042)]
    red events: []
df: ../data/users_feats/U12/1.txt
processing: ../data/users_feats/U12/1.txt  - num events: 8355  - red events: 0
Epoch 000: Loss: 1.485 - in: 175.489 sec.
Epoch 001: Loss: 1.389 - in: 187.514 sec.
  avg_loss: tf.Tensor(1.435772, shape=(), dtype=float32)  - max_loss: tf.Tensor(1.5778865,

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

# axes[1].set_ylabel("Accuracy", fontsize=14)
# axes[1].set_xlabel("Epoch", fontsize=14)
# axes[1].plot(train_accuracy_results)

plt.show()

#### Save model to a file

In [ ]:
model_filepath = 'models/model_lm_v1.hdfs'

tf.keras.models.save_model(
    model,
    model_filepath,
    overwrite=True,
    include_optimizer=True
)